In [0]:
!mkdir ./kaggle

In [3]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"narasimha1997","key":"85681d3d3bad7b31d1397574f7f3aa33"}'}

In [13]:
!kaggle datasets download -d paultimothymooney/blood-cells 

 89%|██████████████████████████████████▌    | 94.0M/106M [00:00<00:00, 48.3MB/s]
100%|█████████████████████████████████████████| 106M/106M [00:00<00:00, 176MB/s]


In [21]:
!ls dataset2-master/images

TEST  TEST_SIMPLE  TRAIN


In [40]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, MaxPool2D, Flatten
from keras.applications import vgg16
from keras.preprocessing import image

samples = 9957
batch_size = 16

#run predictions:
generator = image.ImageDataGenerator(
        rescale = 1./255,
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)

dataset = generator.flow_from_directory(
    shuffle = True,
    batch_size = 32,
    target_size = (80, 80),
    directory = 'dataset2-master/images/TRAIN'
)

def model():
    model = Sequential()
    model.add(Conv2D(80, (3,3), strides = (1, 1), activation = 'relu', input_shape = (80, 80, 3)))
    model.add(Conv2D(64, (3,3), strides = (1, 1), activation = 'relu', input_shape = (80, 80, 3)))
    model.add(MaxPool2D(pool_size = (2,2)))
    model.add(Conv2D(64, (3,3), strides = (1,1), activation = 'relu'))
    model.add(Dropout(0.25))
    model.add(Flatten())

    model.add(Dense(128, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4, activation = 'softmax'))

    model.compile(loss = 'categorical_crossentropy', optimizer = 'adadelta', metrics = ['accuracy'])
    
    return model

nn = model()
nn.fit_generator(dataset, steps_per_epoch = None, epochs = 30, verbose = 1)
nn.save('Model.h5')


Found 9957 images belonging to 4 classes.
Epoch 1/30
312/312 [==============================] - 39s 126ms/step - loss: 1.3999 - acc: 0.2444
Epoch 2/30
253/312 [=======================>......] - ETA: 7s - loss: 1.3742 - acc: 0.2814312/312 [==============================] - 39s 125ms/step - loss: 1.3653 - acc: 0.2930
Epoch 3/30
312/312 [==============================] - 39s 125ms/step - loss: 1.2806 - acc: 0.3882
Epoch 4/30
143/312 [============>.................] - ETA: 21s - loss: 1.1770 - acc: 0.4467312/312 [==============================] - 39s 124ms/step - loss: 1.1187 - acc: 0.4846
Epoch 5/30
312/312 [==============================] - 39s 125ms/step - loss: 0.9508 - acc: 0.5798
Epoch 6/30
113/312 [=========>....................] - ETA: 24s - loss: 0.7852 - acc: 0.6491312/312 [==============================] - 39s 125ms/step - loss: 0.7605 - acc: 0.6595
Epoch 7/30
312/312 [==============================] - 39s 125ms/step - loss: 0.6415 - acc: 0.7262
Epoch 8/30
103/312 [========>....

In [30]:
!du -h Features.npy

622M	Features.npy


In [0]:
from google.colab import files
files.download('Model.h5')

In [56]:
from keras.models import load_model
import numpy
import os
from PIL import Image

model = load_model('Model.h5')

#demo code check for EOSINOPHIL

correct = 0
wrong = 0
total = 0

for file in os.listdir('dataset2-master/images/TRAIN/EOSINOPHIL'):
  image = Image.open('dataset2-master/images/TRAIN/EOSINOPHIL/'+file)
  image = image.resize((80, 80))
  image = numpy.array(image, dtype = 'float32')
  image/=255
  image = image.reshape(1, 80, 80, 3)
  prediction = model.predict(image)
  #print(numpy.argmax(prediction))
  if numpy.argmax(prediction) == 0: correct+=1
  else: wrong+=1
  total+=1

print('EOSINOPHIL :::: Result : ', 'Correct prediction %: ', (correct/total)*100, 'Wrong prediction : %', (wrong/total)*100)


#demo code check for LYMPHOCYTE

correct = 0
wrong = 0
total = 0

for file in os.listdir('dataset2-master/images/TRAIN/LYMPHOCYTE'):
  image = Image.open('dataset2-master/images/TRAIN/LYMPHOCYTE/'+file)
  image = image.resize((80, 80))
  image = numpy.array(image, dtype = 'float32')
  image/=255
  image = image.reshape(1, 80, 80, 3)
  prediction = model.predict(image)
  #print(numpy.argmax(prediction))
  if numpy.argmax(prediction) == 1: correct+=1
  else: wrong+=1
  total+=1

print('LYMPHOCYTE :::: Result : ', 'Correct prediction %: ', (correct/total)*100, 'Wrong prediction : %', (wrong/total)*100)

#demo code check for MONOCYTE

correct = 0
wrong = 0
total = 0

for file in os.listdir('dataset2-master/images/TRAIN/MONOCYTE'):
  image = Image.open('dataset2-master/images/TRAIN/MONOCYTE/'+file)
  image = image.resize((80, 80))
  image = numpy.array(image, dtype = 'float32')
  image/=255
  image = image.reshape(1, 80, 80, 3)
  prediction = model.predict(image)
  #print(numpy.argmax(prediction))
  if numpy.argmax(prediction) == 2: correct+=1
  else: wrong+=1
  total+=1

print('MONOCYTE :::: Result : ', 'Correct prediction %: ', (correct/total)*100, 'Wrong prediction : %', (wrong/total)*100)


#demo code check for NEUTROPHIL

correct = 0
wrong = 0
total = 0

for file in os.listdir('dataset2-master/images/TRAIN/NEUTROPHIL'):
  image = Image.open('dataset2-master/images/TRAIN/NEUTROPHIL/'+file)
  image = image.resize((80, 80))
  image = numpy.array(image, dtype = 'float32')
  image/=255
  image = image.reshape(1, 80, 80, 3)
  prediction = model.predict(image)
  #print(numpy.argmax(prediction))
  if numpy.argmax(prediction) == 3: correct+=1
  else: wrong+=1
  total+=1

print('NEUTROPHIL ::: Result : ', 'Correct prediction %: ', (correct/total)*100, 'Wrong prediction : %', (wrong/total)*100)

EOSINOPHIL :::: Result :  Correct prediction %:  95.83500200240287 Wrong prediction : % 4.1649979975971165
LYMPHOCYTE :::: Result :  Correct prediction %:  99.79863068868305 Wrong prediction : % 0.20136931131695532
MONOCYTE :::: Result :  Correct prediction %:  98.95076674737692 Wrong prediction : % 1.0492332526230832
NEUTROPHIL ::: Result :  Correct prediction %:  87.63505402160864 Wrong prediction : % 12.364945978391356


In [52]:
!ls dataset2-master/images/TRAIN

EOSINOPHIL  LYMPHOCYTE	MONOCYTE  NEUTROPHIL
